# Notebook to generate dbt models from SQL generated using R

This notebook takes the SQL queries generated by the [OHDSI/DataQualityDashboard](https://github.com/OHDSI/DataQualityDashboard/) library and builds a dbt DAG out of it.

The following parameters were used to generate the SQL queries to allow improved performance.

```R
# specify if you want to execute the queries or inspect them ------------------------------------------
sqlOnly <- TRUE # set to TRUE if you just want to get the SQL scripts and not actually run the queries
sqlOnlyIncrementalInsert <- TRUE # set to TRUE if you want the generated SQL queries to calculate DQD results and insert them into a database table (@resultsDatabaseSchema.@writeTableName)
sqlOnlyUnionCount <- 50   # in sqlOnlyIncrementalInsert mode, the number of check sqls to union in a single query; higher numbers can improve performance in some DBMS (e.g. a value of 25 may be 25x faster)
```

Copy the resulting queries into the `sql_dir` folder before executing this notebook.

In [ ]:
from pathlib import Path
import re

In [ ]:
sql_dir = Path('./dqd')
dbt_dir = Path('./dbt')

sql_files = sql_dir.glob('*.sql')
# Remove DDL script as this needs to be run first
sql_files = [f for f in sql_files if not f.stem.endswith('ddlDqdResults')]
ddl_file = sql_dir.joinpath('ddlDqdResults.sql')

pattern_cdm_schema = re.compile(r"hive_metastore.dev_vc\.([a-z_]+)")
pattern_result_schema = re.compile(r"hive_metastore.dev_vc_achilles\.([a-z_]+)")

pattern_insert = re.compile("INSERT.+")
model_template = "\nselect * from {{{{ ref( '{model_name}' ) }}}}\n"


In [ ]:
out = {}
for f in sql_files:
    out[f.stem] = {}
    sql = f.read_text()
    for n, s in enumerate(sql.split(';')):
        s = pattern_insert.sub("", s)
        s = pattern_cdm_schema.sub("{{ source('omop', '\\1') }}", s)
        s = pattern_result_schema.sub("{{ source('cdm_results', '\\1') }}", s)
        if s.strip():
            out[f.stem][f"{f.stem}_{n:04d}"] = s.strip()


In [ ]:
for k, v in out.items():
    dbt_dir.joinpath(k).mkdir(exist_ok=True)
    for k1, v1 in v.items():
        if v1.strip():
            dbt_dir.joinpath(k, f"{k1}.sql").write_text(v1)

    level2_model = 'union all'.join(model_template.format(model_name=k1) for k1 in v)
    dbt_dir.joinpath(k, f"{k}.sql").write_text(level2_model)

final_model = "union all".join(model_template.format(model_name=k) for k in out)
dbt_dir.joinpath('dqdashboard_results.sql').write_text(final_model)